<a href="https://colab.research.google.com/github/denverkim/AICT/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG (Retrieval-Augmented Generation) implementation pipeline

# RAG pipeline using OpenAI + Chroma

In [ ]:
# 1. Install required libraries
!pip install langchain langchain_community langchain_openai openai faiss-cpu pypdf tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.1 MB/s eta 0:00

In [ ]:
# 2. Setup OpenAI API Key
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
# 3. Import libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [ ]:
# 4. Load PDF document
loader = PyPDFLoader("/content/(공고문)차세대융합기술연구원 임원(원장,비상임임원) 공개모집 공고(2025-1).pdf")
documents = loader.load()
print(f"Loaded {len(documents)} document chunks")
print(documents[0])

Loaded 5 document chunks
page_content='차세대융합기술연구원 임원추천위원회 공고 제2025 – 1호 차세대융합기술연구원 임원(원장,비상임임원) 공개모집 공고  차세대융합기술연구원은 경기도와 서울대학교가 공동으로 출연하여 설립한 재단법인이며,        국내 최초·최대 규모의 융합기술 전문연구기관으로서 우리나라 미래 성장 동력 발굴에 노력하고 있습니다. 풍부한 경험과 역량을 갖춘 임원을 공개모집 합니다.2025년  3월  11일                                  차세대융합기술연구원 임원추천위원회 위원장1 임용 직위. 인원. 기간. 주요업무공모직위인원임기주요업무원장(상임이사)1명임명일 ~ 2년·차세대융합기술연구원 대표·기관의 경영 및 사업 전반에 대한 총괄비상임이사4명·사업계획 및 예·결산, 정관 및 주요규정 제·개정 심의·조직 및 기구 등 기타 기관 주요사항 심의 등비상임감사1명·재산 상황 및 업무 감사·이사회의 운영에 관한 사항 감사 등2 법적근거 m 지방자치단체 출자·출연기관의 운영에 관한 법률 제9조, 제10조 m 경기도 출자·출연기관 운영에 관한 기본조례 제3조의5, 제3조의6, 제3조의7 m 차세대융합기술연구원 정관 및 임원추천위원회 규정3 자격요건 m 지역, 연령(미성년자 제외), 성별 : 제한없음 m 병역법 제76조(병역의무 불이행자에 대한 제재)에서 정한 병역의무 불이행 없는자 m 다음 기본자격과 경력요건을 갖춘자' metadata={'producer': 'Hancom PDF 1.3.0.550', 'creator': 'Hwp 2024 13.0.0.2151', 'creationdate': '2025-03-10T12:35:47+09:00', 'author': '과학기술부 지방과학진흥과 서기관 김봉수', 'moddate': '2025-03-10T12:35:47+09:00', 'pdfversion': '1.4', 'source': '/content/(공고문)차세대융합기술연구원 임원(원장,비상임임원) 

In [ ]:
# 5. Split document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Maximum chunk size
    chunk_overlap=50     # Overlap between chunks for context continuity
)
docs = text_splitter.split_documents(documents)
print(f"Total chunks created: {len(docs)}")
print(docs[0])

Total chunks created: 14
page_content='차세대융합기술연구원 임원추천위원회 공고 제2025 – 1호 차세대융합기술연구원 임원(원장,비상임임원) 공개모집 공고  차세대융합기술연구원은 경기도와 서울대학교가 공동으로 출연하여 설립한 재단법인이며,        국내 최초·최대 규모의 융합기술 전문연구기관으로서 우리나라 미래 성장 동력 발굴에 노력하고 있습니다. 풍부한 경험과 역량을 갖춘 임원을 공개모집 합니다.2025년  3월  11일                                  차세대융합기술연구원 임원추천위원회 위원장1 임용 직위. 인원. 기간. 주요업무공모직위인원임기주요업무원장(상임이사)1명임명일 ~ 2년·차세대융합기술연구원 대표·기관의 경영 및 사업 전반에 대한 총괄비상임이사4명·사업계획 및 예·결산, 정관 및 주요규정 제·개정 심의·조직 및 기구 등 기타 기관 주요사항 심의 등비상임감사1명·재산 상황 및 업무 감사·이사회의 운영에 관한 사항 감사 등2 법적근거 m 지방자치단체 출자·출연기관의 운영에 관한 법률' metadata={'producer': 'Hancom PDF 1.3.0.550', 'creator': 'Hwp 2024 13.0.0.2151', 'creationdate': '2025-03-10T12:35:47+09:00', 'author': '과학기술부 지방과학진흥과 서기관 김봉수', 'moddate': '2025-03-10T12:35:47+09:00', 'pdfversion': '1.4', 'source': '/content/(공고문)차세대융합기술연구원 임원(원장,비상임임원) 공개모집 공고(2025-1).pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}


In [ ]:
# 6. Create embeddings & build vector DB (Chroma)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = Chroma.from_documents(docs, embeddings)

In [ ]:
# 7. Define retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
# 8. Initialize LLM (OpenAI)
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

In [ ]:
# 9. Build RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# 10. Run query
# query = "Summarize the main concepts explained in this document."
query = "임용절차는 무엇인가요?"
result = qa_chain.invoke({"query": query})

In [ ]:
# 11. Print results
print("Answer:", result["result"])
print("Number of source documents:", len(result["source_documents"]))
for idx, doc in enumerate(result["source_documents"], 1):
    print(f"\n[{idx}] {doc.page_content[:300]}...\n")

Answer: 임용절차는 다음과 같습니다:

1. 공고 및 접수
2. 임원추천위원회 심사
3. 후보자 추천 (직위별 2배수 내)
4. 임명권자 임명

이 과정에서 3월부터 4월까지 후보자 심사(서류 및 면접 심사)가 진행되며, 4월에 후보자가 추천되고, 4월 21일 이후에 임명이 이루어집니다. 단, 상기 일정은 변동될 수 있습니다.
Number of source documents: 3

[1] 운영에 관한 사항 감사 등2 법적근거 m 지방자치단체 출자·출연기관의 운영에 관한 법률 제9조, 제10조 m 경기도 출자·출연기관 운영에 관한 기본조례 제3조의5, 제3조의6, 제3조의7 m 차세대융합기술연구원 정관 및 임원추천위원회 규정3 자격요건 m 지역, 연령(미성년자 제외), 성별 : 제한없음 m 병역법 제76조(병역의무 불이행자에 대한 제재)에서 정한 병역의무 불이행 없는자 m 다음 기본자격과 경력요건을 갖춘자...


[2] * 부패방지 및 국민권익위원회의 설치와 운영에 관한 법률 제82조(비위면직자등의 업제한)에   해당하는 결격사유에 해당하는 자※ 본인의 범죄경력 등을 개별적으로 확인 후 응모에 문제없음을 확인하여야 함   - 범죄경력조회 방법: http://crims.police.go.kr 또는 인근 경찰서를 방문하여 본인 열람 확인 가능※ 경찰청 범죄경력조회 시 관련근거(경찰청에서 발급사유 요청 시 아래 관련근거 활용)   - 「형의 실효 등에 관한 법률」제6조(범죄경력조회･수사경력조회 및 회보의 제한 등)   - 「지방자치단체 출자･출연 기관의...


[3] 임원추천위원회(사무국 nerodj@snu.ac.kr / 031-888-9022)로 문의바랍니다....

